# BNN MNIST 

In [1]:
from pynq import Overlay, allocate
import numpy as np
import time

ol = Overlay("bnn.bit")
bnn_ip = ol.bnn_0

print("Register map:")
print(bnn_ip.register_map)

print("Dict:")
print(ol.ip_dict)

Register map:
RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  IN_r_1 = Register(IN_r=write-only),
  IN_r_2 = Register(IN_r=write-only),
  ys_1 = Register(ys=write-only),
  ys_2 = Register(ys=write-only)
}
Dict:
{'bnn_0': {'type': 'xilinx.com:hls:bnn:1.0', 'mem_id': 's_axi_CTRL', 'memtype': 'REGISTER', 'gpio': {}, 'interrupts': {}, 'parameters': {'C_M_AXI_DATA_ADDR_WIDTH': '64', 'C_M_AXI_DATA_ARUSER_WIDTH': '1', 'C_M_AXI_DATA_AWUSER_WIDTH': '1', 'C_M_AXI_DATA_BUSER_WIDTH': '1', 'C_M_AXI_DATA_CACHE_VALUE': '"0011"', 'C_M_AXI_DATA_DATA_WIDTH': '32', 'C_M_AXI_DATA_ID_WIDTH': '1', 'C_M_AXI_DATA_PROT_VALUE': '"000"', 'C_M_AXI_DATA_RUSER_WIDTH': '1', 'C_M_AXI_DATA_USER_VALUE': '0x00000000', 'C_M_AXI_DATA_WUSER_WIDTH': 

In [2]:
def binarize(x):
    return np.where(x > 0, 1, -1).astype(np.int16)

In [3]:
def predict_one(inp_vec):
    # inp_vec:(784,)
    # int16 {-1, +1}
    IN_buf[:] = inp_vec
    YS_buf[:] = 0

    IN_buf.flush()
    YS_buf.flush()

    bnn_ip.write(0x10, IN_buf.physical_address)
    bnn_ip.write(0x1c, YS_buf.physical_address)
    bnn_ip.write(0x00, 1)

    while (bnn_ip.read(0x00) & 0x2) == 0:
        pass

    YS_buf.invalidate()
    logits = np.array(YS_buf)
    return int(np.argmax(logits))

In [4]:
def test_batch(X, y):
    num_samples = len(y)
    print_every=1000

    correct = 0
    t0 = time.time()

    for i in range(num_samples):
        x_raw = X[i].astype(np.int16)
        x_bin = binarize(x_raw)
        pred = predict_one(x_bin)
        
        if pred == y[i]:
            correct += 1

        if print_every and (i+1) % print_every == 0:
            print(f"{i+1}/{num_samples} done...")

    t1 = time.time()
    acc = correct / num_samples
    print(f"\naccuracy on {num_samples} samples: {acc:.4f}")
    print(f"correct: {correct}/{num_samples}")
    print(f"total time: {t1 - t0:.3f} s  (~{(t1 - t0)/num_samples*1e3:.3f} ms/sample)")
    return acc

In [5]:
mnist = np.load("mnist_test_data_original.npy", allow_pickle=True)
X = mnist.item().get("data").reshape(10000, 784)
y = mnist.item().get("label")

IN_buf = allocate(shape=(784,), dtype=np.int16)
YS_buf = allocate(shape=(10,), dtype=np.int16)

In [6]:
acc = test_batch(X, y)

1000/10000 done...
2000/10000 done...
3000/10000 done...
4000/10000 done...
5000/10000 done...
6000/10000 done...
7000/10000 done...
8000/10000 done...
9000/10000 done...
10000/10000 done...

HW accuracy on 10000 samples: 0.8939
Correct: 8939/10000
Total time: 47.821 s  (~4.782 ms/sample)
